### Setup Everything
Make sure we are loading all dependencies and have our custom changes for sound

In [1]:
# Load in theano 
from theano.sandbox import cuda
from imp import reload
%matplotlib inline

# Loads the utils function
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

#Import images
from PIL import Image

# Why needed?
from utils import plots


WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN 5110)
Using Theano backend.


In [53]:
# Sets up the pathing
path = "/home/jd/nbs/kdaggledata/"
#path = "/home/jd/nbs/kdaggledata/sample/"

# Additional Path validtions
results_path = path+"results/"
valid_path = path+"valid/"

## Set up sound alert
from IPython.display import Audio, display
from imp import reload
import time

def allDone():
  display(Audio(url='https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav', autoplay=True))

In [3]:
import os, sys, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
from keras.callbacks import ModelCheckpoint

### Setting up the Image Dependencies

In [4]:
# Import our class, and instantiate
from keras import backend as theano
import vgg16bn; reload(vgg16bn)

<module 'vgg16bn' from '/home/jd/FastAi/vgg16bn.py'>

In [5]:
from vgg16 import *
vgg = Vgg16()

### Load images and Setup model

In [6]:
# As large as you can, but no larger than 64 is recommended. 
# If you have an older or cheaper GPU, you'll run out of memory, so will have to decrease this.
batch_size=32
no_of_epochs = 3

In [7]:
batches = vgg.get_batches(path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(valid_path, batch_size=batch_size*4)
vgg.finetune(batches)
vgg.model.optimizer.lr = 0.01

Found 24600 images belonging to 2 classes.
Found 4920 images belonging to 2 classes.


In [ ]:
latest_weights_filename = None
for epoch in range(no_of_epochs):
    print ("Running epoch: %d" % epoch)
    vgg.fit(batches, val_batches, nb_epoch=1)
    latest_weights_filename = 'ft%d.h5' % epoch
    vgg.model.save_weights(results_path+latest_weights_filename)
print ("Completed %s fit operations" % no_of_epochs)

In [ ]:
vgg.model.save_weights(path+'results/jd1.h5')

In [8]:
vgg.model.load_weights(path+'results/jd1.h5')

## Model Messing with

#### Data Augmentation

In [9]:
model = vgg_ft(2)

In [10]:
model.load_weights('/home/jd/nbs/kdaggledata/results/jd1.h5')

In [11]:
layers = model.layers

In [12]:
last_conv_idx = [index for index,layer in enumerate(layers) 
                     if type(layer) is Convolution2D][-1]

In [13]:
last_conv_idx

30

In [14]:
layers[last_conv_idx]

In [15]:
conv_layers = layers[:last_conv_idx+1]
conv_model = Sequential(conv_layers)
# Dense layers - also known as fully connected or 'FC' layers
fc_layers = layers[last_conv_idx+1:]

In [16]:
batches = get_batches(path+'train', shuffle=False, batch_size=batch_size)
val_batches = get_batches(path+'valid', shuffle=False, batch_size=batch_size)

val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

Found 24600 images belonging to 2 classes.
Found 4920 images belonging to 2 classes.


In [ ]:
#Seems to freeze here
val_features = conv_model.predict_generator(val_batches, val_batches.nb_sample)

In [ ]:
trn_features = conv_model.predict_generator(batches, batches.nb_sample)

In [ ]:
save_array(results_path + 'train_convlayer_features.bc', trn_features)
save_array(results_path + 'valid_convlayer_features.bc', val_features)

In [17]:
trn_features = load_array(results_path+'train_convlayer_features.bc')
val_features = load_array(results_path+'valid_convlayer_features.bc')

In [18]:
trn_features.shape

(24600, 512, 14, 14)

In [19]:
def proc_wgts(layer): return [o/2 for o in layer.get_weights()]

In [20]:
opt = RMSprop(lr=0.00001, rho=0.7)

In [21]:
def get_fc_model():
    model = Sequential([
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.8),
        Dense(4096, activation='relu'),
        Dropout(0.8),
        Dense(2, activation='softmax')
        ])

    for l1,l2 in zip(model.layers, fc_layers): l1.set_weights(proc_wgts(l2))

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [22]:
fc_model = get_fc_model()

In [ ]:
fc_model.fit(trn_features, trn_labels, nb_epoch=8, 
             batch_size=batch_size, validation_data=(val_features, val_labels))

In [ ]:
fc_model.save_weights(results_path+'jd_no_dropout.h5')

In [23]:
fc_model.load_weights(results_path+'jd_no_dropout.h5')

### Adding in data Augmentation

In [24]:
gen = image.ImageDataGenerator(rotation_range=15, width_shift_range=0.15, 
                               height_shift_range=0.15, zoom_range=0.15, horizontal_flip=True)

In [25]:
# How do I double the amount of files?
batches = get_batches(path+'train', gen, batch_size=batch_size)

# NB: We don't want to augment or shuffle the validation set
val_batches = get_batches(path+'valid', shuffle=False, batch_size=batch_size)

Found 24600 images belonging to 2 classes.
Found 4920 images belonging to 2 classes.


In [26]:
fc_model = get_fc_model()

In [27]:
for layer in conv_model.layers: layer.trainable = False
# Look how easy it is to connect two models together!
conv_model.add(fc_model)

In [28]:
conv_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
conv_model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=2, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

In [ ]:
conv_model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=3, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

In [ ]:
conv_model.save_weights(results_path + 'jdaug3.h5')
# jdaug1.h5 is loss .2046 acc .9842 and val_loss .1243 val_acc .9904
# jdaug2.h5 is loss .1961 acc .9849 and val_loss .0962 val_acc .9929

In [29]:
conv_model.load_weights(results_path + 'jdaug1.h5')
# jdaug1.h5 is loss .2046 acc .9842 and val_loss .1243 val_acc .9904

### Addings batch norm

In [30]:
conv_layers[-1].output_shape[1:]

(512, 14, 14)

In [31]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(p),
        BatchNormalization(),
        Dense(4096, activation='relu'),
        Dropout(p),
        BatchNormalization(),
        Dense(1000, activation='softmax')
        ]

In [32]:
p=0.6

In [33]:
bn_model = Sequential(get_bn_layers(0.6))

In [35]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_2 (Lambda)                (None, 3, 224, 224)   0           lambda_input_2[0][0]             
____________________________________________________________________________________________________
zeropadding2d_14 (ZeroPadding2D) (None, 3, 226, 226)   0           lambda_2[0][0]                   
____________________________________________________________________________________________________
convolution2d_14 (Convolution2D) (None, 64, 224, 224)  1792        zeropadding2d_14[0][0]           
____________________________________________________________________________________________________
zeropadding2d_15 (ZeroPadding2D) (None, 64, 226, 226)  0           convolution2d_14[0][0]           
___________________________________________________________________________________________

In [36]:
def proc_wgts(layer, prev_p, new_p):
    scal = (1-prev_p)/(1-new_p)
    return [o*scal for o in layer.get_weights()]

In [37]:
for l in bn_model.layers: 
    if type(l)==Dense: l.set_weights(proc_wgts(l, 0.3, 0.6))

In [38]:
bn_model.pop()
for layer in bn_model.layers: layer.trainable=False

In [39]:
bn_model.add(Dense(2,activation='softmax'))

In [40]:
bn_model.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])

In [42]:
bn_model.fit(trn_features, trn_labels, nb_epoch=2, validation_data=(val_features, val_labels))

Train on 24600 samples, validate on 4920 samples
Epoch 1/2
24600/24600 [==============================] - 8s - loss: 0.2955 - acc: 0.8960 - val_loss: 0.2884 - val_acc: 0.8929
Epoch 2/2
24600/24600 [==============================] - 8s - loss: 0.3028 - acc: 0.8953 - val_loss: 0.3273 - val_acc: 0.8797


In [ ]:
#bn_model.load_weights('/home/jd/nbs/kdaggledata/results/bn.h5')

In [43]:
bn_layers = get_bn_layers(0.6)
bn_layers.pop()
bn_layers.append(Dense(2,activation='softmax'))

In [44]:
final_model = Sequential(conv_layers)
for layer in final_model.layers: layer.trainable = False
for layer in bn_layers: final_model.add(layer)

In [45]:
for l1,l2 in zip(bn_model.layers, bn_layers):
    l2.set_weights(l1.get_weights())

In [46]:
final_model.compile(optimizer=Adam(), 
                    loss='categorical_crossentropy', metrics=['accuracy'])

In [47]:
final_model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=2, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Epoch 1/2
24600/24600 [==============================] - 420s - loss: 0.3152 - acc: 0.9432 - val_loss: 0.0786 - val_acc: 0.9768
Epoch 2/2
24600/24600 [==============================] - 420s - loss: 0.1319 - acc: 0.9584 - val_loss: 0.0595 - val_acc: 0.9827


In [48]:
final_model.save_weights(results_path + 'jdfinal1.h5')

In [49]:
allDone()

NameError: name 'alldone' is not defined

In [ ]:
final_model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=2, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

In [ ]:
final_model.save_weights(results_path + 'jdfinal1.h5')

In [ ]:
final_model.load_weights(results_path + 'jdfinal1.h5')

In [67]:
final_model.optimizer.lr=0.00001

In [68]:
final_model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=3, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Epoch 1/3


ValueError: output of generator should be a tuple (x, y, sample_weight) or (x, y). Found: [[[[  31.   31.   30. ...,   23.   32.   29.]
   [  30.   30.   29. ...,   42.  124.   67.]
   [  29.   29.   28. ...,   75.  117.  190.]
   ..., 
   [  43.   48.   45. ...,   94.   84.   86.]
   [  43.   48.   51. ...,  114.  111.  112.]
   [  29.   34.   41. ...,  120.  123.  118.]]

  [[  13.   13.   12. ...,   22.   83.   52.]
   [  12.   12.   11. ...,   45.  180.   94.]
   [  11.   11.   10. ...,   91.  184.  222.]
   ..., 
   [  82.   84.   79. ...,   94.   82.   79.]
   [  79.   82.   83. ...,  115.  110.  106.]
   [  65.   69.   73. ...,  122.  122.  113.]]

  [[   0.    0.    0. ...,   30.  114.   60.]
   [   0.    0.    0. ...,   60.  215.  103.]
   [   0.    0.    0. ...,  116.  226.  233.]
   ..., 
   [  79.   82.   78. ...,   82.   70.   69.]
   [  77.   81.   82. ...,   99.   92.   90.]
   [  61.   65.   70. ...,  100.  101.   91.]]]


 [[[  60.   59.   55. ...,   54.   55.   56.]
 

In [52]:
bn_model.save_weights(results_path + 'jdaugbn1.h5')

In [54]:
allDone()

## Kaggle Submission

In [55]:
#Ensure that you have a testset fed into batches and preds
batches, preds = vgg.test(path+'test', batch_size = batch_size*2)
filenames = batches.filenames

Found 12500 images belonging to 1 classes.


In [56]:
#Test that you can get the predictions AND the files
preds[:5]
filenames[:5]

['test/7807.jpg',
 'test/6818.jpg',
 'test/8924.jpg',
 'test/5985.jpg',
 'test/5468.jpg']

In [57]:
#Save arrays with both predictions and filenames
save_array(path+'results/' + 'test_preds.dat', preds)
save_array(path+'results/' + 'filenames.dat', filenames)

In [58]:
#Load both arrays into variables
preds= load_array(path+'results/' + 'test_preds.dat')
filenames = load_array(path+'results/' + 'filenames.dat')

In [59]:
#ids = np.array([int(f[(f.find('.')):(f.find('.jpg'))]) for f in filenames])
ids = (np.array(f for f in filenames))

In [60]:
isdog= preds[:,1]

In [61]:
# From the np.array cuts the filenames down to just numbers
ids = np.array([int(f[(f.find('/')+1):f.find('.jpg')]) for f in filenames])
#Test
ids[:5]

array([7807, 6818, 8924, 5985, 5468])

In [63]:
isadog= isdog
for n,i in enumerate(isadog):
    if .999<i:
        isadog[n]=.999
    elif .02>i:
        isadog[n]=.001
    else:
        isadog[n]=i
subm = np.stack([ids,isadog], axis=1)
#Test
subm[:5]

array([[  7.8070e+03,   2.0000e-02],
       [  6.8180e+03,   4.8200e-01],
       [  8.9240e+03,   6.2407e-01],
       [  5.9850e+03,   1.1234e-01],
       [  5.4680e+03,   8.8253e-01]])

In [64]:
print ("Raw Predictions: " + str(isadog[:5]))
print ("Mid Predictions: " + str(len(isadog[(isdog < .6) & (isdog > .4)])))
print ("Edge Predictions: " + str(isadog[(isdog == 1) | (isdog == 0)]))

Raw Predictions: [ 0.02    0.482   0.6241  0.1123  0.8825]
Mid Predictions: 1515
Edge Predictions: []


### Submission File Generation

In [65]:
submission_file_name = 'JD5.csv'
np.savetxt(submission_file_name, subm, fmt='%d,%.5f', header='id,label', comments='')

In [66]:
from IPython.display import FileLink
FileLink(submission_file_name)

/home/jd/FastAi/JD5.csv

In [ ]:
allDone()

### wrongs